<a href="https://colab.research.google.com/github/NorrisPau/MachineLearning_Class_NoraPauelsen/blob/main/Predict_sex_with_BERTTopic_probability_vector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
from google.colab import drive

In [6]:
root_path = "/content/gdrive/MyDrive/Machine_Learning_NLP_Nora_Pauelsen_TU_Wien"

In [7]:
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from torch import nn
import matplotlib as plt

# Load Data

In [8]:
#2. Feature Vector X (topic probabilities)
df_probs = pd.read_csv("/content/gdrive/MyDrive/Machine_Learning_NLP_Nora_Pauelsen_TU_Wien/data/processed/df_probs.csv")

#1. Target vector Y (sex)
df_topics = pd.read_csv("/content/gdrive/MyDrive/Machine_Learning_NLP_Nora_Pauelsen_TU_Wien/data/processed/df_topics.csv")

In [9]:
df_probs

,0,1,2,3,4,5,6,7,8,9,...,239,240,241,242,243,244,245,246,247,sum_probabilities
0,0.002899,0.002406,0.003152,0.003018,0.005057,0.004747,0.003091,0.003385,0.004247,0.002223,...,0.006335,0.003545,0.004583,0.005061,0.003559,0.004260,0.003373,0.006101,0.001452,0.792736
1,0.007867,0.007386,0.003879,0.002542,0.003918,0.003415,0.002298,0.002492,0.002432,0.002078,...,0.002893,0.006065,0.003519,0.002741,0.005393,0.003375,0.005193,0.002524,0.002857,0.908086
2,0.002645,0.002816,0.002996,0.022680,0.003528,0.003272,0.006544,0.004964,0.003875,0.002557,...,0.003697,0.003230,0.004766,0.007002,0.002929,0.005803,0.003419,0.004560,0.002395,0.931263
3,0.015836,0.004937,0.003649,0.002089,0.002800,0.002638,0.001931,0.002114,0.001975,0.001542,...,0.002308,0.004138,0.003113,0.002286,0.003329,0.002648,0.004020,0.002110,0.002749,0.794343
4,0.001389,0.001166,0.001549,0.002553,0.002233,0.001655,0.002482,0.002308,0.002018,0.002322,...,0.001939,0.001546,0.001579,0.001925,0.001406,0.001927,0.001439,0.001972,0.000920,0.528361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54453,0.000820,0.000701,0.000798,0.000795,0.001413,0.001736,0.000832,0.000881,0.001705,0.000720,...,0.003096,0.000943,0.001174,0.001442,0.001220,0.001087,0.000920,0.002440,0.000414,0.239767
54454,0.003182,0.002883,0.002324,0.001830,0.001839,0.001693,0.001584,0.001572,0.001291,0.001204,...,0.001485,0.002718,0.002289,0.001623,0.001889,0.002114,0.002704,0.001395,0.002630,0.543139
54455,0.001802,0.008933,0.001046,0.001026,0.001091,0.001148,0.000823,0.000851,0.000803,0.000612,...,0.000972,0.001709,0.001688,0.001146,0.001861,0.001385,0.002230,0.000926,0.001209,0.320321
54456,0.005773,0.002688,0.004719,0.002290,0.007693,0.012233,0.003161,0.002514,0.004011,0.003328,...,0.005816,0.006535,0.003415,0.002699,0.003887,0.002615,0.004305,0.003323,0.001530,0.803854


In [10]:
df_topics

,Profile_text,most_probable_topic,Sex
0,me: would love think kind intellectual: either...,-1,m
1,chef: means. 1. workaholic. 2. love cook regar...,64,m
2,"i'm ashamed much, writing public text online d...",-1,m
3,work library go school. . .,43,m
4,"hey how's going? currently vague profile know,...",-1,m
...,...,...,...
54453,"vibrant, expressive, caring optimist. love peo...",-1,f
54454,i'm nick. never know write myself. i'm sure ha...,-1,m
54455,"hello! enjoy traveling, watching movies, hangi...",-1,m
54456,"""all world balls integrity one take either awa...",-1,m


In [12]:
df_topics["Female"] = np.where(df_topics["Sex"] == "f", 1,0)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(df_probs, df_topics["Female"], test_size=0.33, random_state=42) 

# Baseline: Logistic Regression

In [16]:
import statsmodels.api as sm
logit_model=sm.Logit(df_topics["Female"],df_topics["most_probable_topic"])
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.690420
         Iterations 4
                           Results: Logit
Model:               Logit             Pseudo R-squared:  -0.026    
Dependent Variable:  Female            AIC:               75199.8060
Date:                2022-06-08 15:46  BIC:               75208.7112
No. Observations:    54458             Log-Likelihood:    -37599.   
Df Model:            0                 LL-Null:           -36662.   
Df Residuals:        54457             LLR p-value:       nan       
Converged:           1.0000            Scale:             1.0000    
No. Iterations:      4.0000                                         
--------------------------------------------------------------------
                     Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
--------------------------------------------------------------------
most_probable_topic -0.0038   0.0002 -16.7924 0.0000 -0.0043 -0.0034



# Pytorch Workflow

In [18]:
import torch
import torch.nn as nn

In [16]:
X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test)
X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(y_train)

In [15]:
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

In [17]:
X_train.shape, y_train.shape

(torch.Size([36486, 249]), torch.Size([36486]))

In [1]:
# Make device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

NameError: ignored